In [1]:
import pandas as pd

# Знакомство и предобработка данных

Посмотрим содержимое файла [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA).

In [2]:
df_credits = pd.read_csv('../../datasets/tmdb_5000_credits.csv', low_memory=False)
print(df_credits.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB
None


Посмотрим содержимое файла [tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9)

In [3]:
df_movies = pd.read_csv('../../datasets/tmdb_5000_movies.csv', low_memory=False)
print(df_movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

Объединим общие данные о фильмах и каст фильмов. 
Оставим в датасете только фильмы, которые вышли в "релиз".
Уберём фильмы с пропусками в колонках ['overview', 'genres', 'keywords']

In [4]:
#Объединяем df
df = df_credits.merge(df_movies, left_on=['movie_id', 'title'], right_on=['id', 'title'], how='inner')

#Оставляем только фильмы, вышедшие в релиз.
df = df[df.status == 'Released']

#Убераем фильмы с пропусками в колонках ['overview', 'genres', 'keywords']
df = df.dropna(subset=['overview', 'genres', 'keywords'])

print(f"Количество фильмов, оставшихся после фильтрации: {df.shape[0]}")

Количество фильмов, оставшихся после фильтрации: 4792


## Content Based Filtering (Фильтрация на основе содержания) 

Реализуем алгоритм рекомендации на основе описания фильма (`overview`) и ключевых слов к фильму (`keywords`). 
Объедини тексты этих колонок и проведи предобработку:
* Заменим NaN в описании фильма на пустой символ `''`
* Удалим все английские стоп-слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Рассчитаем матрицу [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильмов.

> Параметр `max_features` в `TfidfVectorizer` равен 10000

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Объединяем тексты колонок с описание фильмов и ключевыми словами.
df['overview_keywords'] = df['overview'] + df['keywords']

#Замена NaN на '' в столбце с описанием фильма
df['overview_keywords'] = df['overview_keywords'].fillna('')

# Исключение неинформотивных слов, редковстречающихся слов, небуквенных символов, привидение текста к нижнему регистру.
# Векторизируем столбец с описанием фильма
vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
tf_idf = vectorizer.fit_transform(df['overview_keywords'])

print('Размер матрицы Tf-Idf:', tf_idf.shape)

Размер матрицы Tf-Idf: (4792, 10000)


Рассчитаем [cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) 
между фильмами. Составим из этой матрицы `pd.DataFrame`. Для дальнейшего удобства, 
колонки и индексы таблицы назовём согласно`id` фильма.

In [6]:
from sklearn.metrics.pairwise import linear_kernel

# Оценка косинусного сходства
cosine_sim = linear_kernel(tf_idf)

# DataFrame из матрицы оценки косинусного сходства с id фильмов.
df_cosine_sim = pd.DataFrame(cosine_sim, index=df.movie_id, columns=df.movie_id)

## Добавление взвешенного рейтинга
Фильмы, которые пользуются большей популярностью и признанием критиков, с большей вероятностью понравятся среднему зрителю.

Для справедливой оценки фильмов возьмем текущую рейтинговую систему IMDB (weighted rating (WR)), которая рассчитывается по формуле:
$$WR = \frac{v}{v + m} ⋅ R + \frac{m}{v + m} ⋅ C$$ 
$v$ - количество голосов \
$m$ - количество голосов для включения в финальную таблицу \
$R$ - средняя оценка \
$C$ - средняя оценка всех фильмов 

Расcчитаем рейтинг для каждого фильма и сохраним его в колонку simple_score.

В качестве параметра  𝑚
> В качестве параметра $m$ выбери 95-й квантиль количества голосов.

In [7]:
# Функция расчёта рейтинга IMDB
def weighted_rating(row):
    v = row['vote_count']
    r = row['vote_average']
    wr = v / (v + m) * r + m / (v + m) * c
    return round(wr, 1)

In [8]:
# Количество голосов для включения в расчёт средней оценки фильмов
m = df['vote_count'].quantile (.95)

# Cредняя оценка фильмов, с количеством голосов более m
c = (df[df.vote_count > m]['vote_average']).mean()

# Запускаем функцию расчёта рейтинга IMDB для каждого фильма 
df['simple_score'] = df.apply(weighted_rating, axis=1)

Сохраним получившиеся датасеты c расстояниями и фильмами в папку [assets](src/assets).

In [9]:
df_cosine_sim.to_csv(r'../assets/distance.csv')
df.to_csv(r'../assets/movies.csv')